# <center>Oil Properties Estimation</center>

### <center>Sept 2014<br>By: Bill Lehr</center>

## Minimum Requirements

Adios 3 requires certain minimum amounts of data for any oil to be included in the library

Crude Oil:
<ul>
<li>API and/or density at a reference temperature</li>
<li>Viscosity (either kinematic or dynamic) at a reference temperature</li>
</ul>

Refined Product or 'Other' oil:
<ul>
<li>API and/or density at a reference temperature</li>
<li>Viscosity (either kinematic or dynamic) at a reference temperature</li>
<li>At least three distillation cuts giving mass or volume fraction at boiling(bubble) point</li>
</ul>

Unlike Adios 2, Adios 3 stores both structural and distillation fractional components <b><u>(see figure 1)</u></b>.
Certain properties are valid for the whole oil while others may be defined for each structural fraction and still others vary for both the distillation and chemical structural fraction.

The SQL oil database needs a complete set of oil data.  If measured data exists, it takes priority over estimated values.  Many of the more complex estimation formulas are based upon Adios 2 calculations and/or the following reference:

<b><i>Characterization and Properties of Petroleum Fractions</b></i><br>
<b>Author:</b> Dr. M. R. Riazi, Professor of Chemical Engineering, Kuwait University<br>
<b>Published:</b> 2005<br>
<b>Publisher:</b> American Society for Testing and Materials (ASTM), International<br>
<b>Stock No.:</b> MNL50<br>

This will be abbreviated further in this document as CPPF.

## Estimation Rules

In [1]:
# just a bit of setup for the following code sections.
import numpy as np

import oil_library
from oil_library.models import Oil, ImportedRecord, KVis

session = oil_library._get_db_session()

ans_mp = session.query(ImportedRecord).filter(ImportedRecord.oil_name == 'ALASKA NORTH SLOPE (MIDDLE PIPELINE)').one()
ans_2002 = session.query(ImportedRecord).filter(ImportedRecord.oil_name == 'ALASKA NORTH SLOPE (2002)').one()
bahia = session.query(ImportedRecord).filter(ImportedRecord.oil_name == 'BAHIA').one()
arabian = session.query(ImportedRecord).filter(ImportedRecord.oil_name == 'ARABIAN MEDIUM, AMOCO').one()
canola = session.query(ImportedRecord).filter(ImportedRecord.oil_name == 'CANOLA OIL (REFERENCE)').one()

print ans_mp
print ans_2002
print bahia
print arabian
print canola


<ImportedRecord('ALASKA NORTH SLOPE (MIDDLE PIPELINE)')>
<ImportedRecord('ALASKA NORTH SLOPE (2002)')>
<ImportedRecord('BAHIA')>
<ImportedRecord('ARABIAN MEDIUM, AMOCO')>
<ImportedRecord('CANOLA OIL (REFERENCE)')>


### 1. Density:

(A) If no density value exists, estimate it from the oil's API using the following equation:

$$
\begin{align}
\rho0_{oil} &= \text{density of the oil at a } T_{ref} \text{of } 288.15^\circ K \,\, (kg/m^3) \cr
&= {141.5 \over 131.5 + API} \cdot 1000 \qquad \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 1)} \cr
\cr
&\text{and solving for API...} \cr
\cr
API &= \left( {141.5 \over \rho0_{oil}} \cdot 1000 \right) - 131.5 \cr
\end{align}
$$

(B) If a density measurement $\rho_0$ at some temperature $T_0$ exists, but no API, then (eq. 1) can be inverted to give an API.  But this can only be done after the density value has been adjusted to be a density at $T_{ref} = 288.15^\circ K (15^\circ C)$.  The density conversion formula for different temperatures is:

$$
\begin{align}
\rho_0 &= \text{measured density} \cr
T_0 &= \text{temperature at which density is measured} \cr
k_{\rho T} &= 0.008 K^{-1} \cr
\cr
\rho_{ref} &= \rho_0 \cdot (1 - k_{\rho T} \cdot (T_{ref} - T_0 )) \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 2)} \cr
\end{align}
$$

<i>(Referenced Source: Adios2)</i>

In [2]:
def estimate_density_from_api(api):
    kg_m_3 = 141.5 / (131.5 + api) * 1000.0
    ref_temp_k = 273.15 + 15.0

    return kg_m_3, ref_temp_k

def estimate_api_from_density(density):
    return (141.5 / density * 1000.0) - 131.5
    
def estimate_density_at_temp(ref_density, ref_temp_k,
                             temperature):
    k_pt = 0.008
    return ref_density / (1 - k_pt * (ref_temp_k - temperature))

def closest_to_temperature(obj_list, temperature):
    '''
        From a list of objects containing a ref_temp_k attribute,
        return the object that is closest to the specified temperature
    '''
    temp_diffs = [(obj, abs(obj.ref_temp_k - temperature))
                  for obj in obj_list
                  if obj.ref_temp_k is not None]
    if len(temp_diffs) > 0:
        return sorted(temp_diffs, key=lambda d: d[1])[0][0]
    else:
        return None


def oil_density_at_temp(imported_rec, temperature, weathering=0.0):
    density_list = [d for d in imported_rec.densities
                    if (d.weathering == weathering)]
    closest_density = closest_to_temperature(density_list, temperature)
    
    if closest_density is not None:
        d_ref, t_ref = (closest_density.kg_m_3,
                        closest_density.ref_temp_k)
    elif imported_rec.api is not None:
        d_ref, t_ref = estimate_density_from_api(imported_rec.api)
    else:
        return None

    return estimate_density_at_temp(d_ref, t_ref, temperature)

for obj in (ans_mp, bahia, arabian):
    print 'oil = ', obj.oil_name
    print 'oil.api = ', obj.api
    print 'oil.densities = ', obj.densities

    if obj.api is not None:
        print 'density(api) = ', estimate_density_from_api(obj.api)
    else:
        print 'density(api) = N/A'

    if len(obj.densities) > 0:
        print 'API(density) = ', estimate_api_from_density(oil_density_at_temp(obj, 288.15))
    else:
        print 'API(density) = N/A'

    print 'density at 288.15K = ', (oil_density_at_temp(obj, 288.15), 288.15)
    print

oil =  ALASKA NORTH SLOPE (MIDDLE PIPELINE)
oil.api =  29.9
oil.densities =  [<Density(886.9 kg/m^3 at 273.15K)>, <Density(876.1 kg/m^3 at 288.15K)>]
density(api) =  (876.7038413878562, 288.15)
API(density) =  30.0112430088
density at 288.15K =  (876.1, 288.15)

oil =  BAHIA
oil.api =  35.2
oil.densities =  []
density(api) =  (848.8302339532095, 288.15)
API(density) = N/A
density at 288.15K =  (848.8302339532095, 288.15)

oil =  ARABIAN MEDIUM, AMOCO
oil.api =  None
oil.densities =  [<Density(875.0 kg/m^3 at 288.15K)>]
density(api) = N/A
API(density) =  30.2142857143
density at 288.15K =  (875.0, 288.15)



### 2. Kinematic Viscosity:

If kinematic viscosity value does not exist, estimate from dynamic viscosity.

$$
\begin{align}
\eta 0_{oil} &= \text{measured dynamic viscosity } (kg/(m \cdot s)) \cr
\rho 0_{oil} &= \text{measured density } (kg/m^3) \cr
\cr
v0_{oil} &= \text{kinematic viscosity } (m^2/s) \cr
&= {\eta 0_{oil} \over \rho 0_{oil} } \qquad \qquad \qquad \qquad \qquad \qquad \boldsymbol{(eq. 3)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

The above is correct, but a bit sparse on detail.  Basically we want to take advantage of any and all measured viscosities that are available.  We will of course prefer the kinematic measurements, but we will want to use any dynamic viscosity measurements that are not redundant.  So the basic program flow will be as follows:

<ul>
<li>Collect all existing measured kinematic viscosities and their associated reference temperatures</li>
<li>Collect any dynamic viscosities measured at temperatures not already represented in the kinematic measurements</li>
<li>Convert the dynamic viscosities into kinematic viscosities using (eq. 3)</li>
<li>Combine our kinematic and converted dynamic viscosities into one aggregate list</li>
</ul>

The formula for viscosity at a temperature is not defined here in Bill's estimation document, but we need to define something.

I believe this is what Adios2 does:

$$
\begin{align}
T &= \text{temperature in } \,^\circ K \cr
k_{v2} &= 5000^\circ K \cr
T_{ref} &= \text{measured reference temperature} \cr
v_{ref} &= \text{measured reference viscosity} \cr
\cr
v_T &= \text{the viscosity at a specified temperature} \cr
&= v_{ref} \cdot exp \left( {k_{v2} \over T} - {k_{v2} \over T_{ref}} \right) \cr
&\text{or alternatively...} \cr
&= v_{ref} \cdot exp \left( k_{v2} \cdot \left( T_{ref} - T \over T \cdot T_{ref} \right) \right) \cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

In [3]:
def dvis_to_kvis(dvis, density):
    return dvis / density

def estimate_kvis_at_temp(temp_k, kvis_ref, ref_temp_k):
    k_v2 = 5000.0
    return kvis_ref * np.exp(k_v2 / temp_k - k_v2 / ref_temp_k)

def estimate_kvis_at_temp2(temp_k, kvis_ref, ref_temp_k):
    k_v2 = 5000.0
    return kvis_ref * np.exp(k_v2 * (ref_temp_k - temp_k) / (temp_k * ref_temp_k))

def oil_aggregate_kvis(imported_rec):
    kvis_list = [(k.ref_temp_k, k.m_2_s)
                 for k in imported_rec.kvis]
    dvis_list = [(d.ref_temp_k,
                  dvis_to_kvis(d.kg_ms,
                               oil_density_at_temp(imported_rec, d.ref_temp_k)))
                 for d in imported_rec.dvis]
    agg = dict(dvis_list)
    agg.update(kvis_list)

    return [KVis(ref_temp_k=i[0], m_2_s=i[1]) 
            for i in agg.iteritems()]

def oil_kvis_at_temp(imported_rec, temp_k, weathering=0.0):
    kvis_list = [kv for kv in oil_aggregate_kvis(imported_rec)
                 if (kv.weathering == weathering)]
    closest_kvis = closest_to_temperature(kvis_list, temp_k)

    if closest_kvis is not None:
        kvis_ref, t_ref = (closest_kvis.m_2_s,
                           closest_kvis.ref_temp_k)
    else:
        return None

    return estimate_kvis_at_temp(temp_k, kvis_ref, t_ref)

for obj in (ans_mp, ans_2002, bahia, arabian):
    print 'oil = ', obj.oil_name
    print 'oil.kvis = ', obj.kvis
    print 'oil.dvis = ', obj.dvis
    print 'aggregate kvis = ', oil_aggregate_kvis(obj)
    print 'kvis at 288k = ', oil_kvis_at_temp(obj, 288.0)
    print

# TODO: Okay, the ANS (2002) oil record has a crazy list of dynamic viscosities
#       This has got to be fixed.

oil =  ALASKA NORTH SLOPE (MIDDLE PIPELINE)
oil.kvis =  []
oil.dvis =  [<DVis(0.034 kg/ms at 273.15K)>, <DVis(0.016 kg/ms at 288.15K)>]
aggregate kvis =  [<KVis(3.83357762995e-05 m^2/s at 273.15K)>, <KVis(1.82627553932e-05 m^2/s at 288.15K)>]
kvis at 288k =  1.84285538212e-05

oil =  ALASKA NORTH SLOPE (2002)
oil.kvis =  []
oil.dvis =  [<DVis(0.0232 kg/ms at 273.0K)>, <DVis(0.0767 kg/ms at 273.0K)>, <DVis(0.614 kg/ms at 273.0K)>, <DVis(4.23 kg/ms at 273.0K)>, <DVis(0.152 kg/ms at 288.0K)>, <DVis(0.625 kg/ms at 288.0K)>]
aggregate kvis =  [<KVis(0.000797266514806 m^2/s at 288.0K)>, <KVis(0.00481776765376 m^2/s at 273.0K)>]
kvis at 288k =  0.000797266514806

oil =  BAHIA
oil.kvis =  [<KVis(1.7e-05 m^2/s at 310.9278K)>]
oil.dvis =  []
aggregate kvis =  [<KVis(1.7e-05 m^2/s at 310.9278K)>]
kvis at 288k =  6.11555602209e-05

oil =  ARABIAN MEDIUM, AMOCO
oil.kvis =  [<KVis(2.31e-05 m^2/s at 288.15K)>, <KVis(1.8e-05 m^2/s at 295.15K)>]
oil.dvis =  [<DVis(0.0202 kg/ms at 288.15K)>, <DVis(0.015

### 3. Oil-Water Surface Tension:

If the imported oil record does not contain a value for surface tension, then we will estimate it from the oil's API:

$$
\begin{align}
\sigma_{o-w} &= \text{oil/water surface tension at } 288.15^\circ K \,\, (N/m) \cr
&= 0.001 \cdot (39 - 0.2571 \cdot API) \qquad \qquad \qquad \qquad \boldsymbol{(eq. 4)} \cr
\end{align}
$$

<i>
Reference: Baker, O. and W. Swerdloff (1956), Calculation of Surface Tensions - Finding the Surface Tension of Hydrocarbon Liquids, Oil Gas J. (2 January 1956) pp. 125
</i>

In [4]:
def o_w_surface_tension_from_api(api):
    if api is not None:
        return 0.001 * (39.0 - 0.2571 * api)
    else:
        return None

for obj in (ans_mp, bahia, arabian):
    print 'oil = ', obj.oil_name
    print 'oil/water surface tension = ', o_w_surface_tension_from_api(obj.api)
    print

oil =  ALASKA NORTH SLOPE (MIDDLE PIPELINE)
oil/water surface tension =  0.03131271

oil =  BAHIA
oil/water surface tension =  0.02995008

oil =  ARABIAN MEDIUM, AMOCO
oil/water surface tension =  None



### 4. Pour Point:

If the imported oil record contains a pour point property then we will simply use it when building the final oil record.<br>
Otherwise, if we have measured molecular weights for the distillation fractions (unusual) then use method <b>(A)</b>.<br>
Otherwise, use method <b>(B)</b>

<b>(A)</b> If molecular weight and mass fractions are given for all the oil fractions $j = 1 \dots jMAX$, then an average molecular weight for the whole oil $\bar M_w$ can be estimated as:

$$
\begin{align}
N &= \text{number of distillation cuts} \cr
jMAX &= 2 (N + 1) \cr
M_{w,j} &= \text{molecular weight of component j} (kg/kmole)\cr
fmass_j &= \text{mass fraction of component j} \cr
\cr
\bar M_w &= \sum_1^{jMAX} M_{w,j} \cdot fmass_j \qquad \qquad \qquad \qquad \boldsymbol{(eq. 5)} \cr
\end{align}
$$

<i>(Note: The calculation for $jMAX$ may seem counterintuitive. It simply states that we sum over all the SARA fractions, each distillation cut represents 1 saturate and 1 aromatic fraction, and that resins and asphaltenes do not have distillation cut data.  So for $N$ distillation cuts, we would calculate $2 N + 2 \rightarrow 2(N + 1)$)</i>

Define $SG = \rho_{oil} / 1000 \cdot kg$ as specific gravity

Then, using CPPF eq. 3.119, our pour point temperature is calculated as:

$$
\begin{align}
T_{API} &= \text{reference temperature for oil kinematic viscosity} \cr
&= 311^\circ K \cr
\cr
T_{pp} &= 130.47 SG^{2.97} \cdot \bar M_w^{0.61235 - 0.47357 SG} \cdot v_{oil}^{0.31 - 0.3283 SG} \cdot T_{API} \qquad \qquad \boldsymbol{(eq. 6)} \cr
\end{align}
$$

<b>(B)</b> Pour point is estimated by reversing the viscosity-temperature correction in Adios2 and assuming that, at the pour point, viscosity is equal to 1 million centistokes.

$$
\begin{align}
c_{v1} &= 5000 K \cr
\cr
T_{pp} &= { c_{v1} \cdot T_{ref} \over c_{v1} - T_{ref} ln(v_{ref}) } \qquad \qquad \qquad \qquad \boldsymbol{(eq. 7)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

In <b>(A)</b>, we reference CPPF eq. 3.119, which seems to not include $T_{API}$ as an input.  Riazi describes this equation as:

$$
\begin{align}
SG &= \text{oil specific gravity} \cr
M &= \text{oil molecular weight} \cr
v_{38(100)} &= \text{oil kinematic viscosity at } 37.8^\circ C (100^\circ F) \cr
\cr
T_P &= \text{pour point (ASTM D 97) in } \,^\circ K \cr
&= 130.47 [SG^{2.970566}] \cdot [M^{(0.61235 - 0.47357 SG)}] \cdot [v_{38(100)}^{(0.310331 - 0.32834 SG)}] \cr
\end{align}
$$

So I think that eq. 6 is not correct.

In <b>(B)</b>, exactly which $(T_{ref}, v_{ref})$ measurement should we use if multiple viscosity measurements exist?  Since we are calculating pour point, which I understand to mean the lowest temperature for which a finite viscosity exists, I believe we should use the lowest measured temperature and its associated viscosity.

Does the viscosity-temperature correction formula in Adios2 define what $c_{v1}$ is?

<center><b>--- End JamesM Comments ---</b></center>

In [5]:
def lowest_temperature(obj_list):
    '''
        From a list of objects containing a ref_temp_k attribute,
        return the object that has the lowest temperature
    '''
    if len(obj_list) > 0:
        return sorted(obj_list, key=lambda d: d.ref_temp_k)[0]
    else:
        return None

def estimate_pour_point_from_ref_temp(ref_temp_k, v_ref):
    c_v1 = 5000.0
    T_pp = (c_v1 * ref_temp_k) / (c_v1 - ref_temp_k * np.log(v_ref))
    
    return T_pp

def get_pour_point_attr(imported_rec):
    if imported_rec.pour_point_min_k is not None:
        return imported_rec.pour_point_min_k
    elif imported_rec.pour_point_max_k is not None:
        return imported_rec.pour_point_max_k
    else:
        return None

# pour point estimation based on (T_ref, v_ref)
# seems a bit inaccurate
for obj in (ans_mp, bahia, arabian):
    print 'oil = ', obj.oil_name
    print 'pour point(min|max) = ', get_pour_point_attr(obj)
    lowest_kvis = lowest_temperature(oil_aggregate_kvis(obj))
    ref_temp_k, v_ref = lowest_kvis.ref_temp_k, lowest_kvis.m_2_s
    print 'pour point (T_ref, v_ref) = ', estimate_pour_point_from_ref_temp(ref_temp_k, v_ref)
    print


oil =  ALASKA NORTH SLOPE (MIDDLE PIPELINE)
pour point(min|max) =  219.15
pour point (T_ref, v_ref) =  175.598251438

oil =  BAHIA
pour point(min|max) =  310.9278
pour point (T_ref, v_ref) =  184.752720919

oil =  ARABIAN MEDIUM, AMOCO
pour point(min|max) =  257.15
pour point (T_ref, v_ref) =  178.394617122



In [6]:
# let's try the estimation based on molecular weights
for obj in (ans_mp_oil,):
    print 'oil = ', obj.name
    for fmass in obj.sara_fractions:
        print fmass

    print
    for mw in obj.molecular_weights:
        print mw

# OK, we have mass fractions for everything,
# but we are missing molecular weights for our resins & asphaltenes
# we will have to punt on this one for now.

NameError: name 'ans_mp_oil' is not defined

### 5. Flash Point:

If measured distillation cut data exists, use method <b>(A)</b>.<br>
Otherwise, use method <b>(B)</b>.

<b>(A)</b> Flash point can be estimated from the first pseudo-component cut:

$$
\begin{align}
T_{cut1} &= \text{the boiling point of the first pseudo-component cut } (^\circ K) \cr
\cr
T_{flsh} &= 117 + 0.69 \cdot T_{cut1} \qquad \qquad \qquad \qquad \boldsymbol{(eq. 8)} \cr
\end{align}
$$

<i>
Reference: Chang A., K. Pashakanti, and Y. Liu (2012), Integrated Process Modeling and Optimization, Wiley Verlag.
</i>

<b>(B)</b> Flash point can be estimated from the API value:

$$
\begin{align}
T_{flsh} &= 457 - 3.34 \cdot API \qquad \qquad \qquad \qquad \boldsymbol{(eq. 9)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

I have some reservations about simply using the first distillation cut boiling point.<br>
It seems that the results could vary wildly depending on the quality of the cut data.

I would be much more confident in a curve fit to the cuts, in which we take the temperature where a certain fraction is evaporated.

<center><b>--- End JamesM Comments ---</b></center>

In [ ]:
def estimate_flash_point_from_bp(ref_temp_k):
    return 117.0 + 0.69 * ref_temp_k

def estimate_flash_point_from_api(api):
    return 457.0 - 3.34 * api

def oil_flash_point(imported_rec):
    if len(imported_rec.cuts) > 0:
        return estimate_flash_point_from_bp(imported_rec.cuts[0].vapor_temp_k)
    elif imported_rec.api is not None:
        return estimate_flash_point_from_api(imported_rec.api)
    else:
        return None

# ANS oil has both an api and distillation cuts.
# For this oil, methods (A) and (B) estimate different results.
# but the difference is only about 7%.  Probably acceptable.
# we should try this on other oils to compare.
for obj in (ans_mp, bahia, arabian):
    print 'oil = ', obj.oil_name
    if obj.api is not None:
        print 'api flash point = ', estimate_flash_point_from_api(obj.api)
    else:
        print 'api = ', obj.api

    if len(obj.cuts) > 0:
        print 'cut flash point = ', estimate_flash_point_from_bp(obj.cuts[0].vapor_temp_k)
    else:
        print 'cuts = ', obj.cuts

    print 'oil flash point = ', oil_flash_point(obj)
    print

### 6. Maximum water fraction of emulsion:

This quantity will be set after the emulsification approach in Adios3 is finalized.  It will vary depending upon the emulsion stability.  For now the following rule will be applied:

If our substance is a crude oil, then $f_{w max} = 0.9$<br>
If our substance is a refined product, then $f_{w max} = 0$

### 7. Bullwinkle (time):

Adios3 needs to know when to initiate emulsification.  The Adios2 development team called this term Bullwinkle.  SINTEF has measured this parameter for some oils, and the new, not yet completed, analysis of emulsification may provide formulas for Bullwinkle.  Bullwinkle may be either a time value (i.e. time delay after which the emulsification formulas are turned on) or a fraction of the oil that needs to evaporate or dissolve before emulsification can start.

Bullwinkle(time) is undefined unless the user explicitly sets a value.  Then it overrides Bullwinkle(fraction) as the determining parameter for the onset of emulsification.

### 8. Bullwinkle (fraction):

<i>
Reference: Adios2
</i>

Bullwinkle is the mass fraction that must evaporate or dissolve before stable emulsification can begin.  This formula will change when we complete the emulsification module.

If $f_{asph} > 0$, then we use method <b>(A)</b><br>
Otherwise, we use method <b>(B)</b><br>

<b>(A)</b> $f_{bull}$ can be calculated from $f_{asph}$:
$$
\begin{align}
f_{bull} &= 0.32 - 3.59 \cdot f_{asph} \qquad \qquad \qquad \boldsymbol{(eq. 10)} \cr
\end{align}
$$

<b>(B)</b> $f_{bull}$ can be calculated from the oil's API:
$$
\begin{align}
f_{bull} &= 0.5762 \cdot log10(API) - 0.6353 \qquad \qquad \boldsymbol{(eq. 11)} \cr
\end{align}
$$

A result of $f_{bull} < 0$ or $f_{bull} > 1$ indicates no emulsification.

In [ ]:
def estimate_bullwinkle_fraction_from_asph(f_asph):
    return 0.32 - 3.59 * f_asph

def estimate_bullwinkle_fraction_from_api(api):
    return 0.5762 * np.log10(api) - 0.6353

def oil_bullwinkle_fraction(imported_rec):
    if (imported_rec.asphaltenes is not None and
            imported_rec.asphaltenes > 0.0):
        return estimate_bullwinkle_fraction_from_asph(imported_rec.asphaltenes)
    elif imported_rec.api is not None:
        return estimate_bullwinkle_fraction_from_api(imported_rec.api)
    else:
        est_api = estimate_api_from_density(oil_density_at_temp(imported_rec, 288.15))
        return estimate_bullwinkle_fraction_from_api(est_api)

for obj in (ans_mp, ans_2002, bahia, arabian):
    print 'oil = ', obj.oil_name
    print 'oil.api = ', obj.api
    print 'oil.asphaltenes = ', obj.asphaltenes
    print 'oil.densities = ', obj.densities

    print 'bullwinkle(fraction) = ', oil_bullwinkle_fraction(obj)
    print


### 9. Adhesion <i>(not currently used by model)</i>:

If the imported oil record contains a valid value for adhesion, we will use that value when building our oil.<br>
Otherwise, $Adh_{oil} = 0.035 kg/m^2$

<i>
Reference: ESTC data
</i>

### 10. Sulphur Mass Fraction <i>(not currently used by model)</i>:

If the imported oil record contains a valid value for sulphur mass fraction, we will use that value when building our oil.<br>
Otherwise, $f_{sulf} = 0$

### 11. Solubility:

If the imported oil record contains a valid value for solubility, we will use that value when building our oil.<br>
Otherwise, $c_{solu} = 0 \, kg/m^3$

### 12. Distillation Cut Boiling Point:

<i>
<b>Note:</b> This and the following properties involve quantities that refer to only certain hydrocarbon groups in the oil.  Adios divides the oil up into $N$ distillation cuts and 4 structure groupings. Two of the structure groupings, resins and asphaltenes, are considered inert with regard to evaporation, biodegradation, and dissolution.  The other two groups, saturates and aromatics, do participate at different rates in these processes.
</i>

<i>
<b>Note:</b> If the culled library data for a particular cut is given at a reduced temperature, this cut and any remaining cuts are considered non-volatile.  The boiling point can be set to an arbitrarily high value.
</i>

<center><b><i>A Note About Data Quality</i></b></center>

Because there are so many calculations that propagate from the distillation cuts, it is important that we interpret them in such a way that results in a reasonable, or at least plausible, distillation curve.  And since we are dealing with records that were manually entered into a database, data entry errors need to be anticipated.

We expect the evaporated fractions to be cumulative with rising temperature.  And if someone incorrectly entered a value when generating the imported oil record, then our cut data could fail to follow this cumulative trend, and our evaporation curve is thrown off.  Some specific errors and their implications are:

- <b>Smaller fraction</b> entered than what it should have been: This will likely cause the fraction to be smaller than the previous one, and the $fmass_i$ value will go negative.
- <b>Bigger fraction</b> entered than what it should have been: This will likely cause the next fraction to be smaller, and the next $fmass_i$ value will go negative.
- <b>Smaller temperature</b> entered than what it should have been: Optimistically the series of $(T_i, fevap_i)$ could be reordered by temperature.  But if we reorder a mistaken temperature, then the fraction will be mistakenly reordered as well, and the error will manifest itself in a similar way as a <b>bigger fraction</b>.

I don't think we can treat this in a naive way.  

- We need to treat the temperatures and their associated fractions as atomic units of data
- Temperatures are required to be in ascending order.  If the temperature of a particular cut is lower than the previous, we throw out the cut.
- Cut fractions might seem to be less consequential.  This will happen only once per error in the series, and we could simply clip the value to be $\geq 0$.  But this will have implications in that the sum of the differences will add up to something bigger than the total evaporated amount, which we don't want.  It's probably better to just throw out the cut.

If no distillation data exists, then we can estimate the distillation data from the oil's API:

First we estimate our lower and upper temperature bounds:

$$
\begin{align}
T_0 &= \text{the lower temperature boundary} \cr
&= 457 - 3.34 \cdot API \qquad \qquad \qquad \qquad \boldsymbol{(eq. 12)} \cr
T_G &= \text{the upper temperature boundary} \cr
&= 1357 - 247.7 \cdot ln(API) \qquad \qquad \qquad \boldsymbol{(eq. 13)} \cr
\end{align}
$$

Next, we would like to generate a set of $N$ temperatures associated with our cuts $T_i$, where $1 \leq i \leq N$.  We will use a default of 5 cuts for this.

$$
\begin{align}
N &= 5 \cr
T_i &= T_0 + T_G \cdot {(i - 1) \over N} \qquad \qquad \qquad \boldsymbol{(eq. 14)} \cr
\end{align}
$$

<i>
Reference: Adios2 and Jones R. (1997), A Simplified Pseudo-component Oil Evaporation Model, Proceedings of the 20th Arctic and Marine Oil Spill Program (AMOP), Vancouver, CA, Vol. 1, pp. 43-62
</i>

<center><b>--- Begin JamesM Comments ---</b></center>

I think we need to clarify $i$ a bit further so there is no confusion here.  $i$ will be used as an index to generate N temperatures.  So $i$ needs to be an integer list of length $N$.  Stating the above equation more precisely along these terms, we might express the following:

$$
\begin{align}
T_i &= T_0 + T_G \cdot {i - 1 \over N} \quad \{ i \in \Bbb Z \,\,|\,\, 1 \leq i \leq N \}\cr
\end{align}
$$

But as a programmer, I like to index my lists starting at zero.  So "I" would state the equation as follows:

$$
\begin{align}
T_i &= T_0 + T_G \cdot {i \over N} \quad \{ i \in \Bbb Z \,\,|\,\, 0 \leq i < N \}\cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

In [ ]:
def estimate_cut_temps_from_api(api, N=5):
    T_0 = 457.0 - 3.34 * api
    T_G = 1357.0 - 247.7 * np.log(api)
    
    return np.array([(T_0 + T_G * i / N)
                     for i in range(N)])

def oil_culled_cuts(imported_rec):
    prev_temp = prev_fraction = 0.0
    for c in imported_rec.cuts:
        if c.vapor_temp_k < prev_temp:
            continue

        if c.fraction < prev_fraction:
            continue

        prev_temp = c.vapor_temp_k
        prev_fraction = c.fraction

        yield c
        

def oil_cut_temps(imported_rec):
    culled_cuts = list(oil_culled_cuts(imported_rec))
    if len(culled_cuts) > 0:
        return np.array([c.vapor_temp_k
                         for c in culled_cuts])
    elif imported_rec.api is not None:
        return estimate_cut_temps_from_api(imported_rec.api)
    else:
        est_api = estimate_api_from_density(oil_density_at_temp(imported_rec, 288.15))
        return estimate_cut_temps_from_api(est_api)

def oil_component_temps(imported_rec):
    cut_temps = oil_cut_temps(imported_rec)
    
    component_temps = np.append([1015.0, 1015.0],
                                zip(cut_temps, cut_temps))
    return np.roll(component_temps, -2)

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    print 'oil.api = ', obj.api
    print 'oil.densities = ', obj.densities
    print 'oil.cut.vapor_temp_k = ', [c.vapor_temp_k for c in obj.cuts]
    print 'oil.cut.fraction = ', [c.fraction for c in obj.cuts]
    print 'cut temperatures = ', oil_cut_temps(obj)
    print 'component temperatures = ', oil_component_temps(obj)
    print


### 13. Initial Component Molecular Weight <i>(required for each distillation cut)</i>:

<i>
Reference: CPPF eq. 2.48 and table 2.6
</i>

The saturate and aromatic component molecular weights can be calculated using the distillation cut temperatures as follows:

$$
\begin{align}
M_{w,sat,i} &= \left( 49.677 \cdot \left[ 6.98291 - ln(1070 - T_i) \right] \right)^{3/2}
\qquad \qquad \qquad \boldsymbol{(eq. 15)} \cr
M_{w,arom,i} &= \left( 44.504 \cdot \left[ 6.911 - ln(1015 - T_i) \right] \right)^{3/2}
\qquad \qquad \qquad \boldsymbol{(eq. 16)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

So that we have a full complement of molecular weights for <u>all</u> components, I would like to see some estimation method for the following:
$$
\begin{align}
M_{w,res} &= \text{molecular weight of our resin component} \cr
M_{w,asph} &= \text{molecular weight of our asphaltene component} \cr
\end{align}
$$

The calculation method for Pour Point (A) needs to have this.  Bill recommends using the following average values:
$$
\begin{align}
M_{w,res} &= 800 \, g/mol \text{ at } 1015^\circ K \cr
M_{w,asph} &= 1000 \, g/mol \text{ at } 1015^\circ K \cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

In [ ]:
def estimate_saturate_mol_wt(boiling_point):
    T_i = np.array(boiling_point)
    return (49.677 * (6.98291 - np.log(1070.0 - T_i))) ** (3.0 / 2.0)

def estimate_aromatic_mol_wt(boiling_point):
    T_i = np.array(boiling_point)
    return (44.504 * (6.911 - np.log(1015.0 - T_i))) ** (3.0 / 2.0)

def estimate_resin_mol_wt():
    return 800.0

def estimate_asphaltene_mol_wt():
    return 1000.0

def estimate_component_mol_wt(boiling_points):
    rho_list = np.append([estimate_resin_mol_wt(),
                          estimate_asphaltene_mol_wt()],
                         zip(estimate_saturate_mol_wt(boiling_points),
                             estimate_aromatic_mol_wt(boiling_points)))
    return np.roll(rho_list, -2)



for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    cut_temps = oil_cut_temps(obj)
    print 'cut temperatures = ', cut_temps
    print 'saturate mol_wt = ', estimate_saturate_mol_wt(cut_temps)
    print 'aromatic mol_wt = ', estimate_aromatic_mol_wt(cut_temps)
    print 'component mol_wt = ', estimate_component_mol_wt(cut_temps)
    print


### 14. Initial Resin and Asphaltene Fractions and Densities:

Resins and asphaltenes are two of the four structural divisions that separate the various hydrocarbons in the oil (the other two are saturates and aromatics).  Resins and asphaltene fractional groupings are NOT broken down further by distillation cuts.

<i>
Reference: Fingas empirical formulas that are based upon analysis of ESTC oil properties database.
</i>

If the imported oil record contains valid values for resins and asphaltenes, we will use those values when building our oil.<br>
Otherwise, we will need to estimate them.

First we define some values $A$ and $B$ that will be used for our component fraction formulas:

$$
\begin{align}
\rho 0_{oil} &= \text{oil aggregate density at } 288.15^\circ K \text{  (from eqs. 1,2)} \cr
v0_{oil} &= \text{oil kinematic viscosity at } 288.15^\circ K (source ???)\cr
\cr
A &= 10 \cdot exp(0.001 \cdot \rho 0_{oil}) \qquad \qquad \qquad \qquad \boldsymbol{(eq. 17)} \cr
B &= 10 \cdot ln(1000 \cdot \rho 0_{oil} \cdot v0_{oil}) \qquad \qquad \qquad \boldsymbol{(eq. 18)}\cr
\end{align}
$$

Then our component fraction formulas:

$$
\begin{align}
f_{res} &= 0.033 \cdot A + 0.00087 \cdot B - 0.74,
\qquad \text{if } f_{res} < 0 \text{ then } f_{res} = 0
\qquad \qquad \qquad \boldsymbol{(eq. 19)} \cr
f_{asph} &= 0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18,
\qquad \text{if } f_{asph} < 0 \text{ then } f_{asph} = 0
\qquad \qquad \boldsymbol{(eq. 20)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

The above formulas describe a simple limit in that the resulting fractions must be greater than zero.  Stating this in simpler mathematical terms:

$$
\begin{align}
f_{res} &= (0.033 \cdot A + 0.00087 \cdot B - 0.74)_{\bot 0} \cr
f_{asph} &= (0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18)_{\bot 0} \cr
\end{align}
$$

I believe that this is not enough to protect us from producing mass fractions that are inconsistent.  If we do not also have an upper limit, then we could end up with a fraction that is greater than 1.0, or a group of fractions that sum to a value greater than 1.0.<br>
In this document, the resin and asphaltene fractions are described first, which implies that we will calculate them before calculating the saturate and aromatic fractions.  This is fine, but the order of calculation should probably be specified.  If we can assume the order of calculation is to first find the resin fraction, and then the asphaltene fraction, then the above formulas including upper and lower limits will look like this:

$$
\begin{align}
f_{res} &= (0.033 \cdot A + 0.00087 \cdot B - 0.74)_{\bot 0}^{\top 1} \cr
f_{asph} &= (0.000014 \cdot A^3 + 0.000004 \cdot B^2 - 0.18)_{\bot 0}^{\top (1 - f_{res})} \cr
\end{align}
$$

<center><b>--- End JamesM Comments ---</b></center>

In [ ]:
def estimate_A_coeff(density):
    return 10.0 * np.exp(0.001 * density)

def estimate_B_coeff(density, viscosity):
    return 10.0 * np.log(1000.0 * density * viscosity)

def estimate_inert_fractions(density, viscosity):
    A = estimate_A_coeff(density)
    B = estimate_B_coeff(density, viscosity)

    f_res = 0.033 * A + 0.00087 * B - 0.74
    f_asph = (0.000014 * A ** 3.0 +
              0.000004 * B ** 2.0 -
              0.18)

    f_res = np.clip(f_res, 0.0, 1.0)
    f_asph = np.clip(f_asph, 0.0, 1.0 - f_res)
    
    return f_res, f_asph

def oil_inert_fractions(imported_rec):
    f_res, f_asph = imported_rec.resins, imported_rec.asphaltenes

    if f_res is None or f_asph is None:
        # if either value is missing from the record,
        # then we toss both of them out and estimate so
        # everything is consistent.
        density = oil_density_at_temp(imported_rec, 288.15)
        viscosity = oil_kvis_at_temp(imported_rec, 288.15)

        f_res, f_asph = estimate_inert_fractions(density,
                                                 viscosity)

    return f_res, f_asph

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    print 'oil.resins = ', obj.resins
    print 'oil.asphaltenes = ', obj.asphaltenes
    obj_density = oil_density_at_temp(obj, 288.15)
    obj_viscosity = oil_kvis_at_temp(obj, 288.15)
    print 'oil density at temp = ', obj_density
    print 'oil viscosity at temp = ', obj_viscosity
    print 'f_res, f_asph = ', estimate_inert_fractions(obj_density,
                                                       obj_viscosity)
    print 'f_res, f_asph = ', oil_inert_fractions(obj)
    print

# TODO: maybe some graphs of this.

### 15. Distillation Cut Mass Fraction:

The distillation cuts are a series of temperatures and mass fractions.  And each temperature $T_i$ is associated with a fractional value $fevap_i$ representing the portion of substance that was evaporated at that temperature. 

For any temperature below about $530^\circ K$, we can assume that the portion of substance that was evaporated contains a mix of saturates and aromatics, and that no resins or asphaltenes have been released.  This mix of saturates and asphaltenes, the quantity that was evaporated at a distillation cut temperature, is what we would like to consider to be our distillation cut mass fractions.

<b>(A)</b> If the following prerequisites are true:

- The culled oil library contains summed (both saturate and aromatic combined) mass fractions or weight %, $fevap_i$, for the distillation cuts.
- At least two distillation cuts exist for which:
  - $T_i < 530^\circ K$
  - $0 \leq fevap_i \leq (1 - (f_{res} + f_{asph}))$
  - $fevap_i \geq fevap_{i-1}$

Then we will use the distillation cut fractional masses that are supplied to generate $fmass_i$.

The distillation cut mass fractions that we get in the imported oil record are cumulative in nature, so it is necessary to collect the differences in the distillation cut mass fractions when generating $fmass_i$

$$
\begin{align}
fmass_i &= {d \over di} \left( fevap_i \right)
\qquad \{ i \in \Bbb Z \,\,|\,\, 0 \leq i \leq N \}
\qquad \qquad \boldsymbol{(eq. 21)} \cr
\end{align}
$$

It is important that the component mass fractions sum up to 1.0, representing the total mass for the substance.  So the following equation needs to be true.

$$
\begin{align}
1 &= f_{res} + f_{asph} + \sum_{i=0}^{N-1} fmass_i \cr
\end{align}
$$

If the fractions do not add up to 1.0, we may need to scale them. Exactly how is unclear.  Which ones will we scale, the inert ones or the volatile ones (or both)?  One possibility, in which we scale only the volatile fractions, is shown below:

$$
\begin{align}
S_{cf} &= \text{cut fractions scaling factor} \cr
\cr
1 &= f_{res} + f_{asph} + \sum_{i=0}^{N-1} fmass_i \cr
1 &= f_{res} + f_{asph} + S_{cf} \left( \sum_{i=0}^{N-1} fmass_i \right) \cr
S_{cf} &= {1 - f_{res} - f_{asph} \over \sum_{i=0}^{N-1} fmass_i } \cr
\end{align}
$$

<b>(B)</b> If distillation cut temperatures were generated by approximation (eq. 18), then it can be assumed that there were no measured mass fractions.  For this we will evenly distribute our fractional masses such that for all distillation cuts:

$$
\begin{align}
fmass_i &= {(1 - (f_{res} + f_{asph})) \over N}
\qquad \qquad \boldsymbol{(eq. 22)} \cr
\end{align}
$$

In [ ]:
def estimate_fmasses_from_cuts(f_res, f_asph, f_evap_i):
    fmass_i = np.array(f_evap_i)
    fmass_i[1:] = np.diff(fmass_i)

    if np.sum(fmass_i) > 0:
        S_cf = (1.0 - f_res - f_asph) / np.sum(fmass_i)
        return fmass_i * S_cf
    else:
        return fmass_i

def estimate_n_fmasses(f_res, f_asph, N=5):
    return np.array([(1.0 - f_res - f_asph) / N] * N)

def oil_fmasses(imported_rec, f_res=None, f_asph=None):
    if f_res is None or f_asph is None:
        f_res, f_asph = oil_inert_fractions(imported_rec)

    culled_cuts = list(oil_culled_cuts(imported_rec))
    if len(culled_cuts) > 0:
        fractions = [c.fraction for c in culled_cuts]
        return estimate_fmasses_from_cuts(f_res, f_asph,
                                          fractions
                                          )
    else:
        return estimate_n_fmasses(f_res, f_asph)

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    print 'oil cut temps = ', oil_cut_temps(obj)
    fractions = [c.fraction for c in oil_culled_cuts(obj)]
    print 'oil.cuts.fractions = ', fractions
    f_res, f_asph = oil_inert_fractions(obj)
    print 'f_res, f_asph = ', f_res, f_asph

    fmasses = estimate_fmasses_from_cuts(f_res, f_asph, fractions)
    print 'sum(fmasses) = ', np.sum(fmasses) + f_res + f_asph

    fmasses = estimate_n_fmasses(f_res, f_asph)
    print 'N fmasses = ', fmasses
    print 'sum(N fmasses) = ', np.sum(fmasses) + f_res + f_asph
    
    fmasses = oil_fmasses(obj)
    print 'oil_fmasses = ', fmasses
    print 'sum(oil_fmasses) = ', np.sum(fmasses) + f_res + f_asph
    print


### 16. Component Density <i>(required for each distillation cut)</i>:

<i>
Reference: CPPF eq. 2.13 and table 9.6
</i>

Initial density estimate for both resin and asphaltene fractional componens is set to:

$$
\begin{align}
\rho_{res} &= \rho_{asph} = 1100 \, kg/m^3 \cr
\end{align}
$$

Then we define Watson characterization factors for saturates and aromatics:

$$
\begin{align}
K_{arom,w} &= \text{Watson characterization factor for aromatics} \cr
&= 10 \cr
K_{sat,w} &= \text{Watson characterization factor for saturates} \cr
&= 12 \cr
\end{align}
$$

Then we apply the appropriate Watson characterization to calculate an initial trial estimate for each density component:

$$
\begin{align}
\rho try_{arom,i} &= 1000 \cdot {\root 3 \of T_i \over K_{arom,w} }
\qquad \qquad \boldsymbol{(eq. 23)} \cr
\rho try_{sat,i} &= 1000 \cdot {\root 3 \of T_i \over K_{sat,w} }
\qquad \qquad \boldsymbol{(eq. 24)} \cr
\end{align}
$$

This should be a reasonable estimate.  However, the average density (fractionally weighted average of the cut densities) must match the measured value or the value from approximation (eqs. 1, 2).<br>
To find the scaling factor between our densities and the aggregate density, we do the following:

$$
\begin{align}
j &= \text{index representing all oil components } \{ j \in \Bbb Z \} \cr
jMAX &= \text{index of the last oil component} \cr
&= 2N + 2 \cr
fmass0_j &= \text{the fractional mass of our jth component} \cr
\cr
Cf_{dens} &= { \rho 0_{oil} \over \sum_{j=1}^{jMAX} fmass0_j \cdot \rho try_j}
\qquad \qquad \boldsymbol{(eq. 25)} \cr
\end{align}
$$

Then we evenly apply this scaling factor to our trial densities to come up with densities that are consistent with the aggregate density.

$$
\begin{align}
\rho_j = Cf_{dens} \cdot \rho try_j \qquad \qquad \boldsymbol{(eq. 26)} \cr
\end{align}
$$

<center><b>--- Begin JamesM Comments ---</b></center>

There is a bit of a catch 22 here in regards to scaling the densities.  In order to scale our densities, we need component mass fractions.  And in order to get our component mass fractions, we need component specific gravities and therefore densities.  Here is what we might be able to do.

- calculate our trial densities
- calculate trial specific gravities based on trial densities
- calculate our component mass fractions based on trial specific gravities
- scale our component densities based on component mass fractions, trial densities, and the overall oil density

It may improve the top-down organization of these estimations if we put the density scaling in its own section.

<center><b>--- End JamesM Comments ---</b></center>

In [ ]:
def estimate_trial_densities(boiling_points, watson_factor):
    return 1000.0 * boiling_points ** (1.0 / 3.0) / watson_factor

def estimate_saturate_densities(boiling_points):
    return estimate_trial_densities(boiling_points, 12)

def estimate_aromatic_densities(boiling_points):
    return estimate_trial_densities(boiling_points, 10)

def estimate_resin_density():
    return 1100.0

def estimate_asphaltene_density():
    return 1100.0

def estimate_component_densities(boiling_points):
    rho_list = np.append([estimate_resin_density(),
                          estimate_asphaltene_density()],
                         zip(estimate_saturate_densities(boiling_points),
                             estimate_aromatic_densities(boiling_points)))
    return np.roll(rho_list, -2)

def oil_component_densities(imported_rec):
    cut_temps = oil_cut_temps(imported_rec)
    return estimate_component_densities(cut_temps)
    

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    cut_temps = oil_cut_temps(obj)
    print 'oil cut temps = ', cut_temps
    print 'aromatic densities = ', estimate_aromatic_densities(cut_temps)
    print 'saturate densities = ', estimate_saturate_densities(cut_temps)
    print 'trial densities from oil = ', oil_component_densities(obj)
    print


### 17. Component Specific Gravity:

Specific Gravity of a substance is the ratio of density of the substance to the density of water at a specified temperature.<br>

<i>
Reference: https://en.wikipedia.org/wiki/Specific_gravity
</i>

For simplicity, we will make the assumption that our water density is $1000 \, kg/m^3$.  So our estimation of the specific gravity of our oil components will be:

$$
\begin{align}
SG_j &= \rho_j / 1000 \qquad \qquad \boldsymbol{(eq. 27)} \cr
\end{align}
$$

In [ ]:
def estimate_specific_gravity(density):
    return density / 1000.0

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    rho_list = oil_component_densities(obj)
    print 'trial densities = ', rho_list
    print 'molecular weights = ', estimate_specific_gravity(rho_list)
    print


### 18-19. Initial Saturate and Aromatic Mass Fractions <i>(required for each distillation cut)</i>:

<i>
References:
  <ul>
    <li>CPPF eqs. 2.114, 2.115, 3.77 and 3.78</li>
    <li>Huang, E K., Characterization and Thermodynamic Correlations for Undefined Hydrocarbon Mixtures, Ph.D. Dissertation, Pennsylvania State University, University Park, PA, 1977</li>
  </ul>
</i>

Once we have either measured or estimated distillation cut mass fractions available to us, we can then compute our saturate mass fractions using the boiling point, specific gravity, and molecular weight of our saturate components:

Using Riazi, we perform a number of intermediate calculations to finally come up with the estimation for saturate component fractional mass.

First is a dimensionless parameter $I$, that was first used by Huang to correlate hydrocarbon properties.  This is necessary to calculate the saturate refractive index $n$.<br>

Next is the calculation of refractive index $n$.  This is necessary to calculate $m$.

The next intermediate calculation $m$ can be best described as a hydrocarbon grouping parameter, Riazi describes $m$ as a parameter that "not only separates paraffins and aromatics but also identifies various hydrocarbon types", and is (AFAIK) based on an observed correlation of refractive index and Molecular weight.

$$
\begin{align}
T_i &= \text{distillation cut boiling point} \cr
M_{w,sat,i} &= \text{molecular weight of our saturate component} \cr
SG_{sat, i} &= \text{specific gravity of saturate component} \cr
\cr
I &= \text{hydrocarbon characterization parameter} \cr
&= 0.3773 T_i^{-0.02269} SG_{sat,i}^{0.9182} \qquad \qquad \boldsymbol{(eq. 28)} \cr
\cr
n &= \text{refractive index of our saturate component at } 20^\circ C \cr
&= \left( 1 + 2 I \over 1 - I \right)^{1/2} \qquad \qquad \qquad \qquad \boldsymbol{(eq. 29)} \cr
\cr
m &= \text{hydrocarbon grouping parameter} \cr
&= M_{w,sat,i}(n - 1.475) \qquad \qquad \qquad \boldsymbol{(eq. 30)} \cr
\end{align}
$$

And finally, after these intermediate calculations are completed, we can estimate the saturate component fractional mass.


$$
\begin{align}
fmass_i &= \text{distillation cut fractional mass} \cr
\cr
f_{sat,i} &= (fmass_i \cdot (2.24 - 1.98 \cdot SG_{sat,i} - 0.009 \cdot m))_{\bot 0}^{\top fmass_i}
\qquad \qquad \boldsymbol{(eq. 31)} \cr
\end{align}
$$

After our saturate mass fraction has been computed, we compute our aromatic mass fraction simply as:

$$
\begin{align}
f_{arom,i} &= fmass_i - f_{sat,i} \qquad \qquad \qquad \boldsymbol{(eq. 32)} \cr
\end{align}
$$

In [ ]:
def hydrocarbon_characterization_param(specific_gravity, temp_k):
    T_i = temp_k
    SG_i = specific_gravity
    return 0.3773 * T_i ** (-0.02269) * SG_i ** (0.9182)

def refractive_index(hc_char_param):
    I = hc_char_param
    return ((1 + 2 * I) / (1 - I)) ** (1.0 / 2.0)

def hydrocarbon_grouping_param(mol_wt, specific_gravity, temp_k):
    I = hydrocarbon_characterization_param(specific_gravity, temp_k)
    n = refractive_index(I)

    return mol_wt * (n - 1.475)

def estimate_saturate_mass_fraction(fmass_i, mol_wt, specific_gravity, temp_k):
    SG_sat_i = specific_gravity
    m = hydrocarbon_grouping_param(mol_wt, SG_sat_i, temp_k)
    
    f_sat_i = fmass_i * (2.24 - 1.98 * SG_sat_i - 0.009 * m)

    return np.clip(f_sat_i, 0.0, fmass_i)

def oil_component_mass_fractions(imported_rec):
    cut_temps = oil_cut_temps(imported_rec)

    f_res, f_asph = oil_inert_fractions(imported_rec)
    fmass_i = oil_fmasses(imported_rec, f_res, f_asph)

    sat_mask = np.array(range(len(cut_temps))) * 2

    sat_temps = oil_component_temps(imported_rec)[sat_mask]
    sat_mol_wts = estimate_component_mol_wt(cut_temps)[sat_mask]
    sat_rhos = estimate_component_densities(cut_temps)[sat_mask]
    sat_SGs = estimate_specific_gravity(sat_rhos)

    f_sat_i = estimate_saturate_mass_fraction(fmass_i,
                                              sat_mol_wts,
                                              sat_SGs,
                                              sat_temps)
    f_arom_i = fmass_i - f_sat_i

    f_all = [n for l in zip(f_sat_i, f_arom_i) for n in l] + [f_res, f_asph]
    
    return np.array(f_all)

for obj in (ans_mp, ans_2002, bahia, arabian, canola):
    print 'oil = ', obj.oil_name
    print 'oil.cuts.fractions = ', [c.fraction for c in obj.cuts]
    print 'oil.cuts.vapor_temp_k = ', [c.vapor_temp_k for c in obj.cuts]
    print 'component mass fractions = ', oil_component_mass_fractions(obj)
    print
